In [1]:
# CatBoost

!pip install seaborn catboost -q

import numpy as np
import pandas as pd

In [2]:
path = "C:/Users/Kate/Downloads/train.csv"
train = pd.read_csv(path)

In [3]:
cat_features = ["model", "car_type", "fuel_type"]
targets = ["target_class", "target_reg"]
features2drop = ["car_id"] 

filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

print("cat_features", cat_features)
print("num_features", len(num_features))
print("targets", targets)

for c in cat_features:  # Избавлеямся от NaN'ов
    train[c] = train[c].astype(str)

cat_features ['model', 'car_type', 'fuel_type']
num_features 11
targets ['target_class', 'target_reg']


In [4]:
from sklearn.model_selection import train_test_split

X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["target_reg"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

model = CatBoostRegressor(
    cat_features=cat_features, eval_metric="RMSE"
)

model.fit(
    X_train,
    y_train,
    verbose=500,
    plot=False,
)

print(model.best_score_)

Learning rate set to 0.045195
0:	learn: 17.1862834	total: 13.4ms	remaining: 13.4s
500:	learn: 7.8920293	total: 6.07s	remaining: 6.04s
999:	learn: 6.0539782	total: 12.2s	remaining: 0us
{'learn': {'RMSE': 6.053978188487934}}


In [45]:
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

# Тюнинг гиперпараметров модели

cbr = CatBoostRegressor(
        depth=4,
        iterations=5000,
        learning_rate=0.09,   
        cat_features=cat_features,
        colsample_bylevel=0.098,
        subsample=0.95,)

cbr.fit(
        X_train,
        y_train,
        verbose=500,
        plot=False)


print(cbr.best_score_)

0:	learn: 17.4468389	total: 1.78ms	remaining: 8.88s
500:	learn: 9.9624447	total: 683ms	remaining: 6.13s
1000:	learn: 8.9463974	total: 1.37s	remaining: 5.48s
1500:	learn: 8.1432940	total: 2.06s	remaining: 4.81s
2000:	learn: 7.4925844	total: 2.87s	remaining: 4.3s
2500:	learn: 6.9422285	total: 3.63s	remaining: 3.62s
3000:	learn: 6.4819532	total: 4.36s	remaining: 2.9s
3500:	learn: 6.0339779	total: 5.11s	remaining: 2.19s
4000:	learn: 5.6590537	total: 5.85s	remaining: 1.46s
4500:	learn: 5.3730022	total: 6.53s	remaining: 724ms
4999:	learn: 5.0874696	total: 7.23s	remaining: 0us
{'learn': {'RMSE': 5.087469568640063}}


In [65]:
# LightGBM

!pip install lightgbm -q

import lightgbm as lgb

df = pd.read_csv("train.csv")
cat_cols = ["car_type", "fuel_type", "model"]
drop_cols = ["car_id", "target_reg", "target_class"]

X = df.drop(drop_cols, axis=1)
y = df["target_class"]

In [66]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cat_cols:
    X[col] = le.fit_transform(X[col])

le_tar = LabelEncoder()
y = le_tar.fit_transform(y)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
from lightgbm import Dataset

train_data = Dataset(
    X_train,
    y_train,
    categorical_feature=cat_cols,
    free_raw_data=False,
)

val_data = Dataset(
    X_test,
    y_test,
    categorical_feature=cat_cols,
    free_raw_data=False,
)

In [69]:
from lightgbm import LGBMClassifier, train ,LGBMRegressor


reg = LGBMRegressor(metric="RMSE")

reg.fit(
    X_train,
    y_train,
    eval_set=[
        (X_test, y_test),
    ],
)
print(reg.best_score_)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1634
[LightGBM] [Info] Number of data points in the train set: 1869, number of used features: 13
[LightGBM] [Info] Start training from score 3.867844
defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('rmse', 1.066804001969111)])})


In [70]:
# Тюнинг гиперпараметров модели

reg = LGBMRegressor(
    n_estimators=40,
    cat_feature=[0, 1, 2],
    metric="RMSE")

reg.fit(
    X_train,
    y_train,
    eval_set=[
        (X_test, y_test),
    ],
)

print(reg.best_score_)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000437 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1636
[LightGBM] [Info] Number of data points in the train set: 1869, number of used features: 13
[LightGBM] [Info] Start training from score 3.867844
defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('rmse', 1.0586094452310433)])})


In [13]:
# XGBoost
!pip install xgboost -q
import xgboost as xgb
import warnings; warnings.filterwarnings("ignore")

In [14]:
trainDF = pd.read_csv("C:/Users/Kate/Downloads/train.csv")
drop_cols = ['car_id', 'target_reg', 'target_class']
cat_cols = ['car_type', 'fuel_type', 'model']

X = trainDF.drop(drop_cols, axis=1)
y = trainDF['target_reg']

 
for col in cat_cols:
    X[col] = X[col].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
from sklearn.metrics import mean_poisson_deviance, mean_squared_error

reg = xgb.XGBRegressor(tree_method="hist",
                       eval_metric = mean_poisson_deviance,
                       enable_categorical=True,
                       n_estimators=30, n_jobs=-1
)
reg.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=10,
        verbose=3)

y_pred = reg.predict(X_test, iteration_range=(0, reg.best_iteration + 1))
mean_squared_error(y_test, y_pred) ** 0.5

[0]	validation_0-rmse:15.73818	validation_0-mean_poisson_deviance:4.83291
[3]	validation_0-rmse:13.04271	validation_0-mean_poisson_deviance:3.25132
[6]	validation_0-rmse:12.64687	validation_0-mean_poisson_deviance:3.07632
[9]	validation_0-rmse:12.65160	validation_0-mean_poisson_deviance:3.07671
[12]	validation_0-rmse:12.67780	validation_0-mean_poisson_deviance:3.08867
[15]	validation_0-rmse:12.75195	validation_0-mean_poisson_deviance:3.13961
[18]	validation_0-rmse:12.88637	validation_0-mean_poisson_deviance:3.20034


12.621295288019933

In [16]:
# Тюнинг гиперпараметров модели

reg = xgb.XGBRegressor(tree_method="hist",
                       eval_metric = mean_poisson_deviance,
                       enable_categorical=True,
                       n_estimators=40, 
                       n_jobs=-1,
                       min_child_weight=32,
                       max_bin=128,
                       reg_lambda=275,)

reg.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=10,
        verbose=3)

y_pred = reg.predict(X_test, iteration_range=(0, reg.best_iteration + 1))
mean_squared_error(y_test, y_pred) ** 0.5

[0]	validation_0-rmse:16.97257	validation_0-mean_poisson_deviance:5.67316
[3]	validation_0-rmse:14.55563	validation_0-mean_poisson_deviance:4.06194
[6]	validation_0-rmse:13.44591	validation_0-mean_poisson_deviance:3.44482
[9]	validation_0-rmse:12.88900	validation_0-mean_poisson_deviance:3.16361
[12]	validation_0-rmse:12.50357	validation_0-mean_poisson_deviance:2.97651
[15]	validation_0-rmse:12.27803	validation_0-mean_poisson_deviance:2.87298
[18]	validation_0-rmse:12.14974	validation_0-mean_poisson_deviance:2.82255
[21]	validation_0-rmse:12.04237	validation_0-mean_poisson_deviance:2.77504
[24]	validation_0-rmse:12.01053	validation_0-mean_poisson_deviance:2.77109
[27]	validation_0-rmse:11.96352	validation_0-mean_poisson_deviance:2.75434
[30]	validation_0-rmse:11.95310	validation_0-mean_poisson_deviance:2.75635
[33]	validation_0-rmse:11.94365	validation_0-mean_poisson_deviance:2.75827
[36]	validation_0-rmse:11.91017	validation_0-mean_poisson_deviance:2.74676
[39]	validation_0-rmse:11.906

11.90601931806027